## Bayesian Neural Network Modeling
Shyam Parikh

In [290]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
from tensorflow.keras.layers import Input, Dense, Dropout
import tensorflow_probability.python.distributions as tfd

In [291]:
import pandas
from datetime import datetime,timedelta
import yfinance as yf
from pandas_datareader import data as pdr


In [292]:
START_DATE = (datetime.today() - timedelta(days = 1 * 365)).strftime('%Y-%m-%d') # data start date
END_DATE = datetime.today().strftime('%Y-%m-%d') # data end date
data = pd.read_csv('basePortfolioStocks.csv')
tickers = list(data["Unnamed: 0"])
print(tickers)


['BKNG', 'CDNS', 'CPRT', 'CRM', 'GE', 'META', 'MKTX', 'NVDA', 'ULTA', 'WST']


In [293]:
yf.pdr_override()
tickersList = pdr.get_data_yahoo(tickers, START_DATE, END_DATE)['Adj Close']
tickersLogRetList = np.log(tickersList).diff().dropna()
tickerSum_stats = pd.DataFrame({'Mean':tickersLogRetList.mean(), 'SD': tickersLogRetList.std(), 'Skewness':tickersLogRetList.skew(), 'Excess Kurtosis':tickersLogRetList.kurtosis() , 'RR ratio':(tickersLogRetList.mean()/tickersLogRetList.std())})
tickerSum_stats = tickerSum_stats.sort_values(by='RR ratio', ascending=False)

[*********************100%***********************]  10 of 10 completed


In [294]:
tickerSum_stats

,Mean,SD,Skewness,Excess Kurtosis,RR ratio
CPRT,0.001144,0.018734,0.051885,1.451866,0.061058
CDNS,0.001389,0.022847,0.094824,1.148242,0.060814
GE,0.001280,0.021355,-0.698628,3.028517,0.059951
ULTA,0.001003,0.020736,-0.082191,7.185621,0.048362
MKTX,0.001161,0.025204,-0.537582,6.312962,0.046052
BKNG,0.000785,0.023612,-0.526901,0.816887,0.033237
NVDA,0.000863,0.037792,0.069617,0.648879,0.022832
CRM,0.000210,0.028456,0.233205,1.423688,0.007372
META,0.000151,0.039496,-0.682291,13.403290,0.003815
WST,-0.000098,0.026776,-0.128653,5.256649,-0.003648


In [295]:
data = []
for i in tickers:
    data.append(pdr.get_data_yahoo(i, START_DATE, END_DATE))
data[0]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-04-18 00:00:00-04:00,2210.000000,2226.179932,2173.000000,2199.310059,2199.310059,329400
2022-04-19 00:00:00-04:00,2204.810059,2252.989990,2204.810059,2239.379883,2239.379883,339700
2022-04-20 00:00:00-04:00,2260.000000,2284.250000,2243.770020,2251.800049,2251.800049,265500
2022-04-21 00:00:00-04:00,2316.070068,2337.899902,2224.280029,2239.320068,2239.320068,389800
2022-04-22 00:00:00-04:00,2226.000000,2263.959961,2209.820068,2213.870117,2213.870117,301800
...,...,...,...,...,...,...
2023-04-11 00:00:00-04:00,2581.810059,2588.679932,2561.060059,2566.469971,2566.469971,236700
2023-04-12 00:00:00-04:00,2587.860107,2587.919922,2534.520020,2547.250000,2547.250000,276200
2023-04-13 00:00:00-04:00,2569.000000,2631.100098,2560.500000,2629.629883,2629.629883,334600


In [296]:
data[0] = data[0].drop(['Open','High','Low','Close','Volume'],axis = 1)

## Preprocessing All of the Data

for x in data:
    # Assume you want to predict whether the next hour's closing price will be higher or lower than the current hour's closing price 
    x['returns'] = np.log(x['Adj Close']).diff()
    x['direction'] = np.where(x['returns'] > 0, 1, 0)
    


In [297]:
# Load data
# Preprocess data
scaler = MinMaxScaler()
data[0]['scaled_price'] = scaler.fit_transform(data[0]['Adj Close'].values.reshape(-1, 1))


In [298]:
# Define train and test sets
train_size = int(len(data[0]) * 0.8)
train_data = data[0].iloc[:train_size]
test_data = data[0].iloc[train_size:]

In [299]:
def bayesian_model():
    input_layer = Input(shape=(1,))
    hidden_layer_1 = Dense(32, activation='relu')(input_layer)
    dropout_1 = Dropout(0.2)(hidden_layer_1)
    hidden_layer_2 = Dense(16, activation='relu')(dropout_1)
    dropout_2 = Dropout(0.2)(hidden_layer_2)
    output_layer = Dense(1)(dropout_2)
    model = Model(inputs=input_layer, outputs=tfd.Normal(output_layer, 1))
    return model
model = bayesian_model()


ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: tfp.distributions.Normal("Normal", batch_shape=[?, 1], event_shape=[], dtype=float32)

In [ ]:
# Define negative log likelihood loss function
def negative_log_likelihood(y_true, y_pred):
    return -y_pred.log_prob(y_true)

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=neg_log_likelihood)


NameError: name 'neg_log_likelihood' is not defined

In [ ]:
history = model.fit(train_data[:, 0], train_data[:, 0], epochs=100, batch_size=32, validation_split=0.2)


2/2 [==============================] - 0s 13ms/step - loss: 0.9199


In [ ]:
import yfinance as yf

def get_current_price(ticker):
    stock  = yf.Ticker(ticker)
    price = (stock.history(period = '1d')['Close'][0])
    print(price)
    return (price)

In [ ]:
msft = yf.Ticker("MSFT")

# get all stock info
hist = msft.history(period="1d")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-04-18 00:00:00-04:00,291.570007,291.76001,287.01001,288.369995,20149400,0.0,0.0
